# SOU Exploratory Analyis & File Creation

### Libraries

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pdfplumber
import re

# Personal Lib for this project
from tools import *

In [15]:
def merge_and_pop(lst:list):
    '''merges and pops the name'''
    if ',' in lst[1]:
        last, first = lst[1].split(',')
        last, first = last.lower(), first.lower()
        lst[1] = first.capitalize() + ' ' + last.capitalize()
    else:
        lst[1] = lst[1] + ' ' + lst[2]
        lst.pop(2)
    return lst



### Testing on One File
- TODO:
    Compile everything into one function

In [16]:
# Path to for rep file
pdf_path = "../data/pdf/March20.pdf"

with pdfplumber.open(pdf_path, pages = [1]) as pdf:
    # Extract text from each page
    text = ""
    for page in pdf.pages:
        text += page.extract_text();

# Grab the text and figure out use case

In [17]:
matches = re.findall(r'Southern Oregon \d+..(#[\s\S]+?..TM Team)', text, re.DOTALL)
matches_date = re.findall(r'Date: \s*(........)', text)
matches

['# Player GS MIN FG 3PT FT ORB-DRB REB PF A TO BLK STL PTS\n20 Baird, Morgan * 16 7-14 1-2 8-8 1-3 4 0 0 1 0 0 23\n10 McIntyre, Bridgette * 32 5-10 4-7 0-0 1-3 4 0 2 1 0 3 14\n12 Schmerbach, Emma * 30 4-12 0-1 6-6 1-1 2 3 0 2 0 0 14\n01 McIntyre, Meghan * 34 2-11 0-0 1-4 1-4 5 3 8 4 0 2 5\n23 Robbins, Clara * 23 0-5 0-0 0-0 0-5 5 3 0 0 0 1 0\n22 Scheppele, Sierra 25 2-4 2-3 0-0 0-1 1 0 0 2 0 2 6\n32 DiGiulio, Eliza 11 1-2 0-0 2-2 1-2 3 2 1 0 0 1 4\n13 Wright, Keeley 9 0-1 0-0 1-2 1-4 5 1 1 0 0 0 1\n05 Scurlock, Makayla 14 0-1 0-0 0-0 0-1 1 1 0 0 0 0 0\n33 Williams, Mallory 6 0-1 0-1 0-0 0-2 2 0 0 0 0 0 0\nTM Team']

In [18]:
stats = matches[0]
stats = stats.strip().split(sep = '\n')
split_stats = list(map(str.split, stats))

opponent = get_opponent(text)
title = split_stats[0] + ['Opponent'] + ['Date']

player_stats = {key: [] for key in title}

players = split_stats[1:-1]
players = list(map(merge_and_pop, players))
players = list(map(non_start, players))
players

stat_dict = {key: [] for key in title}

for player in players:

    for stat, data in zip(title, player):
        stat_dict[stat].append(data)
    stat_dict['Opponent'].append(opponent)
    stat_dict['Date'].append(matches_date[0])

# Proof of Concept
#pd.DataFrame.from_dict(stat_dict)

In [19]:
def folder_to_csv(folder_path:str, test:bool = False):
    
    if test:
        folder = [folder_path]
    else:
        folder = folder_to_list(folder_path)

    i = 0 # used to track the number of files

    for file in folder:
        print(i, file)
        with pdfplumber.open(file, pages= [1]) as pdf:
            
            # Extract text from each page
            text = ""
            for page in pdf.pages:
                text += page.extract_text();
        matches = re.findall(r'Southern Oregon \d+..(#[\s\S]+?..TM T)', text, re.DOTALL)
        matches_date = re.findall(r'Date: \s*(........)', text)

        stats = matches[0]
        stats = stats.strip().split(sep = '\n')
        split_stats = list(map(str.split, stats))

        opponent = get_opponent(text)
        title = split_stats[0] + ['Opponent'] + ['Date']

        players = split_stats[1:-1]
        players = list(map(merge_and_pop, players))
        players = list(map(non_start, players))
        players

        if i == 0:
            stat_dict = {key: [] for key in title}

        for player in players:

            for stat, data in zip(title, player):
                stat_dict[stat].append(data)
            stat_dict['Opponent'].append(opponent)#new
            stat_dict['Date'].append(matches_date[0])#new
 
        if len(stat_dict['Date']) != len(stat_dict['Player']):
            print(matches_date, 'Length of PTS and Player do not match')
        i += 1

    return (stat_dict)

# Testing Area

In [20]:
len(folder_to_list('../data/pdf/'))

35

In [21]:
x = folder_to_csv('../data/pdf')

0 ../data/pdf/Jan03.pdf
1 ../data/pdf/March1.pdf
2 ../data/pdf/Jan17.pdf
3 ../data/pdf/March4.pdf
4 ../data/pdf/March20.pdf
['03/20/25'] Length of PTS and Player do not match
5 ../data/pdf/March22.pdf
['03/22/25'] Length of PTS and Player do not match
6 ../data/pdf/Jan04.pdf
['01/04/25'] Length of PTS and Player do not match
7 ../data/pdf/Jan10.pdf
['01/10/25'] Length of PTS and Player do not match
8 ../data/pdf/Jan11.pdf
['01/11/25'] Length of PTS and Player do not match
9 ../data/pdf/Dec14.pdf
['12/14/24'] Length of PTS and Player do not match
10 ../data/pdf/Feb7.pdf
['02/07/25'] Length of PTS and Player do not match
11 ../data/pdf/Nov23.pdf
['11/23/24'] Length of PTS and Player do not match
12 ../data/pdf/Dec06.pdf
['12/06/24'] Length of PTS and Player do not match
13 ../data/pdf/Dec07.pdf
['12/07/24'] Length of PTS and Player do not match
14 ../data/pdf/Feb1.pdf
['02/01/25'] Length of PTS and Player do not match
15 ../data/pdf/Feb14.pdf
['02/14/25'] Length of PTS and Player do not 

In [22]:
for key in x:
    print(key, len(x[key]))

# 397
Player 397
GS 397
MIN 397
FG 397
3PT 397
FT 397
ORB-DRB 397
REB 397
PF 397
A 397
TO 397
BLK 397
STL 397
PTS 397
Opponent 407
Date 402


In [23]:
tst = folder_to_csv('../data/pdf/March20.pdf', test = True)
for key in tst:
    print(key, len(tst[key]))

0 ../data/pdf/March20.pdf
['03/20/25'] Length of PTS and Player do not match
# 10
Player 10
GS 10
MIN 10
FG 10
3PT 10
FT 10
ORB-DRB 10
REB 10
PF 10
A 10
TO 10
BLK 10
STL 10
PTS 10
Opponent 20
Date 15
